In [1]:
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
submit_path = datetime.now().strftime('../submits/%d-%H%M.csv')
submit_path

'../submits/31-2256.csv'

In [11]:
model = pickle.load(open('../dataset/lightgbmmodel.pkl', 'rb'))

In [4]:
test_feature = pd.read_pickle('../dataset/test/all_feature.pkl')
test_feature.shape

(978298, 265)

In [5]:
use_columns = [
    x for x in test_feature.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid', 'orderlabel'
    ]
]

In [12]:
result = model.predict_proba(test_feature[use_columns])[:, 1]

In [13]:
test_feature['prob'] = result

In [14]:
test_feature.sort_values('prob', ascending=False, inplace=True)

In [15]:
submit = test_feature.drop_duplicates(['orderid'])[['orderid', 'roomid', 'prob']]

In [16]:
submit['orderid'] = 'ORDER_' + submit['orderid'].astype(str)

In [17]:
submit['predict_roomid'] = 'ROOM_' + submit.roomid.astype(str)

In [18]:
submit.to_csv(submit_path, index=False, columns=['orderid', 'predict_roomid'])

In [19]:
!wc -l $submit_path

34633 ../submits/31-2256.csv


In [20]:
!head $submit_path

orderid,predict_roomid
ORDER_139093,ROOM_4164603
ORDER_107558,ROOM_11202563
ORDER_118946,ROOM_79925911
ORDER_113074,ROOM_7978779
ORDER_117651,ROOM_9970699
ORDER_113124,ROOM_89529411
ORDER_139034,ROOM_7575543
ORDER_120517,ROOM_6534649
ORDER_108401,ROOM_4730123


In [21]:
# test_feature.orderid.unique().shape